In [2]:
!pip install pickle5
!pip install pydicom

     |████████████████████████████████| 133kB 7.4MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219250 sha256=8d4f3c1706de778ace89da6bfcfa3300708e0abbaa94eb621a4665d0f56d86a5
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
     |████████████████████████████████| 1.9MB 3.5MB/s 


In [3]:
ls

sample_data/


In [4]:
import os
from PIL import Image
from torch.utils.data import Dataset
import pandas as pd
import torch
import torchvision
import numpy as np
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms
import albumentations as A
#from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys  #to set local import folder
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc  #debug memory leaks in matplotlib
import csv #read in description files
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#show model design parameters with torchsummary
import torchsummary
from torchsummary import summary
from torch import FloatTensor
from torch import tensor
from torch.utils.data import DataLoader
from google.colab import files
import pydicom #to read dicom files
from pydicom import dcmread
import pickle5 as pickle; #generic storage of image array
from torchvision.io import read_image
from torchvision import transforms  #get normalization functions
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
class CustomImageDataset(): #Dataset):
    def __init__(self, img_dir,category=[],file_count=1,file_list =[],transform=None, target_transform=None):
        #self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.category = category
        self.file_count = file_count
        self.file_list = file_list
        self.transform = transform
        self.target_transform = target_transform
        self.category_name =''

    def image_normalize(image):
        #replace with the more tensor friendly normalize once tensor shapes confirmed
        image = image/65535.0
        return image

    def __len__(self):
        return self.file_count #len(self.file_list) #99 #len(self.img_labels)

    def __getitem__(self, index):        
        fname = self.file_list[index]
        #get label and pull category 
        text_tokens = fname.split(sep='_')
        label_class = text_tokens[3] #get the label token in 4th position
        self.category_name =  label_class.upper() 


        full_file_name = os.path.join(self.img_dir,self.category_name,fname)
        image = pickle.load( open( full_file_name, "rb" ) )
        image = image.astype(float) #using patch images
        
        #VERIFY THE IMAGES ARE ALL THE SAME 3x244x244
        shapes = image.shape
        assert (shapes[0] == 3),"Image slice error: {0}".format(fname)
        assert (shapes[1] == 244), print('Image row error: ',fname)
        assert (shapes[2] == 244), print('Image column error: ',fname)
        #if (shapes[0] != 3 and shapes[1] != 244 and shapes[2]!= 244):

        #Normalize the data to 0,1 from 2^16
        image = image/65535.0 #image_normalize(image)

        #test out numeric label
        if (label_class in 'Normal'):
            label = 0
        elif (label_class in 'Actionable'):
            print('!!!!! ACTIONABLE PASSED THROUGH')
            stop()
            label = 1
        elif (label_class in 'Benign'):
            label = 1
        else: # (label_class in 'Cancer'):
            label = 2


        #print(full_file_name)


        #read_image(img_path)
        #label = self.img_labels.iloc[idx, 1]
        #if self.transform:
        #    image = self.transform(image)
        #if self.target_transform:
        #    label = self.target_transform(label)
        sample = {"image": image, "label": label}
            #sample = file_name
        return image, label

In [6]:
### Enable GPU, if present
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    !nvidia-smi -L
    !nvidia-smi 
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')

data_dir = 'drive\\Shareddrives\\DBTdata\\DBT_DATA\\IMG_ARRAYS\\'
patch_dir = 'drive/Shareddrives/DBTdata/DBT_DATA/CLASSIFICATION_TASK/TRAINING_PATCHES'
#model_dir = 
#tensorboard_dir = 

GPU 0: Tesla T4 (UUID: GPU-09aebec7-a0a9-9357-6d36-e00bfbc27916)
Tue Jun  8 00:08:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [7]:
cd content/

[Errno 2] No such file or directory: 'content/'
/content


In [8]:
ls drive/Shareddrives/DBTdata/DBT_DATA/CLASSIFICATION_TASK/TRAINING_PATCHES/

ACTIONABLE/  BENIGN/  CANCER/  NORMAL/


In [9]:
use_patch_files = 1
if (use_patch_files == 0):
    raw_files = os.listdir(data_dir)
    print('found #files: ',len(raw_files))
else: #patches broken up into directories
    category_folders = os.listdir(patch_dir) #raw_files = os.listdir(data_dir) #print('found #files: ',len(raw_files))    

if (0):
    #create fake patches for now
    patch_dict = {}
    for counter,filename in enumerate(raw_files):
        #load full array
        full_filename = os.path.join(data_dir,filename)
        img_data = pickle.load( open( full_filename, "rb" ) )
        patch_data = img_data[0:3,:,:]
        patch_dict[counter]= patch_data
        print(full_filename,np.shape(patch_data))
        if (counter > 0):
            break
        
### SKIP ACTIONABLE FOLDER
### Remove actionable folder item from list, since we're no longer using that data
category_folders.remove('ACTIONABLE')

In [10]:
for ii in category_folders:
    print(ii)
    flist = os.listdir(os.path.join(patch_dir,ii))

temp = flist[0].split(sep='_')
print(temp[3])

NORMAL
BENIGN
CANCER
Cancer


In [11]:
#generate full file list for use in indexing the dataloader
#this replaces the older loader, which was only inputting Normal
full_file_list = [] #store the full filename of every file
full_category_name = []
for category_folder in category_folders:
            print('------------- ',category_folder)
            file_list = os.listdir(os.path.join(patch_dir,category_folder))
            cat_count = 0
            for file_name in file_list:
                cat_count = cat_count + 1
                full_category_name.append(category_folder)
                full_file_list.append(file_name)
                print(os.path.join(category_folder,file_name))
full_file_count = len(full_file_list)
print(len(full_file_list))
full_file_list[0]

Streaming output truncated to the last 5000 lines.
NORMAL/DBT-P03062_DBT-S02786_lmlo_Normal_s39_cx415_cy1079_244_244.pickle
NORMAL/DBT-P03062_DBT-S02786_rmlo_Normal_s13_cx1439_cy1086_244_244.pickle
NORMAL/DBT-P03081_DBT-S05220_lmlo_Normal_s45_cx389_cy998_244_244.pickle
NORMAL/DBT-P03081_DBT-S05220_lcc_Normal_s42_cx347_cy1319_244_244.pickle
NORMAL/DBT-P03081_DBT-S05220_rmlo_Normal_s26_cx1496_cy1032_244_244.pickle
NORMAL/DBT-P03081_DBT-S05220_rcc_Normal_s24_cx1532_cy1240_244_244.pickle
NORMAL/DBT-P03079_DBT-S00277_lmlo_Normal_s70_cx507_cy1204_244_244.pickle
NORMAL/DBT-P03079_DBT-S00277_rcc_Normal_s39_cx1460_cy1158_244_244.pickle
NORMAL/DBT-P03079_DBT-S00277_rmlo_Normal_s47_cx1386_cy1146_244_244.pickle
NORMAL/DBT-P03079_DBT-S00277_lcc_Normal_s11_cx478_cy1344_244_244.pickle
NORMAL/DBT-P03088_DBT-S02585_rcc_Normal_s41_cx1432_cy1282_244_244.pickle
NORMAL/DBT-P03088_DBT-S02585_lmlo_Normal_s19_cx524_cy1207_244_244.pickle
NORMAL/DBT-P03088_DBT-S02585_lcc_Normal_s3_cx491_cy1372_244_244.pickle
NO

'DBT-P03261_DBT-S05042_rmlo_Normal_s43_cx1582_cy1161_244_244.pickle'

##Dataset

In [12]:
datasetK = CustomImageDataset(img_dir=patch_dir, category = full_category_name, file_count=len(full_file_list), file_list = full_file_list, transform=None, target_transform=None)

#Data loader for training and validation sets
def get_loaders(batch_size, tf, num_workers=4, pin_memory=True, validation_split=0.2):
    dataset = datasetK
    dataset_size = len(dataset) #split dataset into train and val sets
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split*dataset_size))
    
    train_indices, val_indices = indices[split:], indices[:split]
    train_sample = SubsetRandomSampler(train_indices)
    val_sample = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, sampler=train_sample)
    val_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, sampler=val_sample)

    return train_loader, val_loader


In [ ]:
#****Adapted from David's ResNet
# Hyperparameters etc.   
LEARNING_RATE = 0.005
device = "cuda" #if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 200
NUM_WORKERS = 2
IMAGE_HEIGHT = 344 
IMAGE_WIDTH = 344  
PIN_MEMORY = True
NUM_WORKERS = 2

def main():
    #Set up the model
    transformation = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()]) #Transform images
    model = torchvision.models.resnet50().to(device)             #Choose model
    loss_fn = nn.CrossEntropyLoss()                              #Choose loss
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)  #Choose optimizer
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.5) #Choose learning rate schedule
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    #validation_saved_index,test_saved_index,saved_flist=pickle.load( open( index_file, "rb" ))       
    train_loader, val_loader = get_loaders(BATCH_SIZE,transformation,NUM_WORKERS,PIN_MEMORY, 0.2)                  #Get Train and Validation sets
    #check_accuracy(val_loader, model, device=DEVICE)
    #scaler = torch.cuda.amp.GradScaler()
    steps = 0; print_every = 10; running_loss = 0; train_losses, test_losses = [],[]
    
    #Run the model
    for epoch in range(NUM_EPOCHS): #should probably reformat this to print once per epoch 
        for inputs, labels in train_loader:
            inputs.float()
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs.float())
            loss = loss_fn(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if steps % print_every == 0:
                test_loss = 0
                acc = 0.00
                model.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device),labels.to(device)
                        logps = model.forward(inputs.float())
                        batch_loss = loss_fn(logps, labels)
                        test_loss += batch_loss.item()
                        
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        acc += torch.mean(equals.type(torch.DoubleTensor)).item()
                train_losses.append(running_loss/len(train_loader))
                test_losses.append(test_loss/len(val_loader))                    
                print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Val loss: {test_loss/len(val_loader):.3f}.. "
                      f"Val accuracy: {acc/len(val_loader):.3f}")
                running_loss = 0
                model.train()
                


if __name__ == "__main__":
    main()

25557032
Epoch 1/200.. Train loss: 0.994.. Val loss: 6.710.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.661.. Val loss: 7.321.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.269.. Val loss: 7.861.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.228.. Val loss: 12.054.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.343.. Val loss: 20.700.. Val accuracy: 0.019
Epoch 1/200.. Train loss: 0.278.. Val loss: 21.468.. Val accuracy: 0.058
Epoch 1/200.. Train loss: 0.267.. Val loss: 18.821.. Val accuracy: 0.023
Epoch 1/200.. Train loss: 0.362.. Val loss: 26.080.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 0.351.. Val loss: 41.374.. Val accuracy: 0.076
Epoch 1/200.. Train loss: 0.200.. Val loss: 21.337.. Val accuracy: 0.028
Epoch 2/200.. Train loss: 0.303.. Val loss: 47.824.. Val accuracy: 0.111
Epoch 2/200.. Train loss: 0.221.. Val loss: 32.306.. Val accuracy: 0.116
Epoch 2/200.. Train loss: 0.311.. Val loss: 25.021.. Val accuracy: 0.000
Epoch 2/200.. Train loss: 0.298.. Val loss: 2

In [ ]:
#training loss decreased to 0.00 but val loss stuck at 20ish and val accuracy at 0.02 or so

In [ ]:
ls

CLASSIFICATION_TASK/  IMG_ARRAYS/  TEST_DATA/  TRAINING_DATA/  VALIDATION/


In [ ]:
cd DBT_DATA

/content/drive/.shortcut-targets-by-id/1GVZJSs7K9GFU34p1HH_F8xaTEOJHc0NE/DBT_DATA


In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 0.005
device = "cuda" #if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 200
NUM_WORKERS = 2
IMAGE_HEIGHT = 344 
IMAGE_WIDTH = 344  
PIN_MEMORY = True
NUM_WORKERS = 2

def main():
    #Set up the model
    transformation = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()]) #Transform images
    model = torchvision.models.resnet50().to(device)             #Choose model
    loss_fn = nn.CrossEntropyLoss()                              #Choose loss
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)  #Choose optimizer
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5) #Choose learning rate schedule
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    #validation_saved_index,test_saved_index,saved_flist=pickle.load( open( index_file, "rb" ))       
    train_loader, val_loader = get_loaders(BATCH_SIZE,transformation,NUM_WORKERS,PIN_MEMORY, 0.2)                  #Get Train and Validation sets
    #check_accuracy(val_loader, model, device=DEVICE)
    #scaler = torch.cuda.amp.GradScaler()
    steps = 0; print_every = 10; running_loss = 0; train_losses, test_losses = [],[]
    
    #Run the model
    for epoch in range(NUM_EPOCHS): #should probably reformat this to print once per epoch 
        for inputs, labels in train_loader:
            inputs.float()
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs.float())
            loss = loss_fn(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if steps % print_every == 0:
                test_loss = 0
                acc = 0.00
                model.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device),labels.to(device)
                        logps = model.forward(inputs.float())
                        batch_loss = loss_fn(logps, labels)
                        test_loss += batch_loss.item()
                        
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        acc += torch.mean(equals.type(torch.DoubleTensor)).item()
                train_losses.append(running_loss/len(train_loader))
                test_losses.append(test_loss/len(val_loader))                    
                print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Val loss: {test_loss/len(val_loader):.3f}.. "
                      f"Val accuracy: {acc/len(val_loader):.3f}")
                running_loss = 0
                model.train()
                


if __name__ == "__main__":
    main()

25557032
Epoch 1/200.. Train loss: 1.429.. Val loss: 6.639.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.373.. Val loss: 7.391.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.265.. Val loss: 8.586.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.282.. Val loss: 13.172.. Val accuracy: 0.023
Epoch 1/200.. Train loss: 0.329.. Val loss: 13.359.. Val accuracy: 0.030
Epoch 1/200.. Train loss: 0.232.. Val loss: 20.541.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.263.. Val loss: 48.684.. Val accuracy: 0.060
Epoch 1/200.. Train loss: 0.228.. Val loss: 74.648.. Val accuracy: 0.093
Epoch 1/200.. Train loss: 0.158.. Val loss: 58.737.. Val accuracy: 0.104
Epoch 1/200.. Train loss: 0.233.. Val loss: 40.779.. Val accuracy: 0.088
Epoch 2/200.. Train loss: 0.212.. Val loss: 28.660.. Val accuracy: 0.000
Epoch 2/200.. Train loss: 0.116.. Val loss: 30.329.. Val accuracy: 0.000
Epoch 2/200.. Train loss: 0.311.. Val loss: 19.575.. Val accuracy: 0.019
Epoch 2/200.. Train loss: 0.255.. Val loss: 2

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 0.005
device = "cuda" #if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 50
NUM_EPOCHS = 200
NUM_WORKERS = 2
IMAGE_HEIGHT = 224 
IMAGE_WIDTH = 224  
PIN_MEMORY = True
NUM_WORKERS = 2

def main():
    #Set up the model
    transformation = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()]) #Transform images
    model = torchvision.models.resnet50().to(device)             #Choose model
    loss_fn = nn.CrossEntropyLoss()                              #Choose loss
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)  #Choose optimizer
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5) #Choose learning rate schedule
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    #validation_saved_index,test_saved_index,saved_flist=pickle.load( open( index_file, "rb" ))       
    train_loader, val_loader = get_loaders(BATCH_SIZE,transformation,NUM_WORKERS,PIN_MEMORY, 0.2)                  #Get Train and Validation sets
    #check_accuracy(val_loader, model, device=DEVICE)
    #scaler = torch.cuda.amp.GradScaler()
    steps = 0; print_every = 10; running_loss = 0; train_losses, test_losses = [],[]
    
    #Run the model
    for epoch in range(NUM_EPOCHS): #should probably reformat this to print once per epoch 
        for inputs, labels in train_loader:
            inputs.float()
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs.float())
            loss = loss_fn(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if steps % print_every == 0:
                test_loss = 0
                acc = 0.00
                model.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device),labels.to(device)
                        logps = model.forward(inputs.float())
                        batch_loss = loss_fn(logps, labels)
                        test_loss += batch_loss.item()
                        
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        acc += torch.mean(equals.type(torch.DoubleTensor)).item()
                train_losses.append(running_loss/len(train_loader))
                test_losses.append(test_loss/len(val_loader))                    
                print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Val loss: {test_loss/len(val_loader):.3f}.. "
                      f"Val accuracy: {acc/len(val_loader):.3f}")
                running_loss = 0
                model.train()
                


if __name__ == "__main__":
    main()

25557032
Epoch 1/200.. Train loss: 1.232.. Val loss: 6.913.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.214.. Val loss: 7.480.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 0.204.. Val loss: 8.931.. Val accuracy: 0.000
Epoch 2/200.. Train loss: 0.194.. Val loss: 10.414.. Val accuracy: 0.000
Epoch 2/200.. Train loss: 0.181.. Val loss: 15.642.. Val accuracy: 0.008
Epoch 2/200.. Train loss: 0.177.. Val loss: 39.835.. Val accuracy: 0.041
Epoch 2/200.. Train loss: 0.163.. Val loss: 65.855.. Val accuracy: 0.078
Epoch 3/200.. Train loss: 0.152.. Val loss: 21.425.. Val accuracy: 0.018
Epoch 3/200.. Train loss: 0.191.. Val loss: 22.336.. Val accuracy: 0.028
Epoch 3/200.. Train loss: 0.141.. Val loss: 28.992.. Val accuracy: 0.057
Epoch 4/200.. Train loss: 0.174.. Val loss: 18.596.. Val accuracy: 0.008
Epoch 4/200.. Train loss: 0.135.. Val loss: 29.059.. Val accuracy: 0.017
Epoch 4/200.. Train loss: 0.157.. Val loss: 26.360.. Val accuracy: 0.035
Epoch 4/200.. Train loss: 0.146.. Val loss: 2

In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 0.5
device = "cuda" #if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 50
NUM_EPOCHS = 200
NUM_WORKERS = 2
IMAGE_HEIGHT = 224 
IMAGE_WIDTH = 224  
PIN_MEMORY = True
NUM_WORKERS = 2

def main():
    #Set up the model
    transformation = transforms.Compose([transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)), transforms.ToTensor()]) #Transform images
    model = torchvision.models.resnet50().to(device)             #Choose model
    loss_fn = nn.CrossEntropyLoss()                              #Choose loss
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)  #Choose optimizer
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5) #Choose learning rate schedule
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    #validation_saved_index,test_saved_index,saved_flist=pickle.load( open( index_file, "rb" ))       
    train_loader, val_loader = get_loaders(BATCH_SIZE,transformation,NUM_WORKERS,PIN_MEMORY, 0.2)                  #Get Train and Validation sets
    #check_accuracy(val_loader, model, device=DEVICE)
    #scaler = torch.cuda.amp.GradScaler()
    steps = 0; print_every = 10; running_loss = 0; train_losses, test_losses = [],[]
    
    #Run the model
    for epoch in range(NUM_EPOCHS): #should probably reformat this to print once per epoch 
        for inputs, labels in train_loader:
            inputs.float()
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs.float())
            loss = loss_fn(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if steps % print_every == 0:
                test_loss = 0
                acc = 0.00
                model.eval()
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(device),labels.to(device)
                        logps = model.forward(inputs.float())
                        batch_loss = loss_fn(logps, labels)
                        test_loss += batch_loss.item()
                        
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        acc += torch.mean(equals.type(torch.DoubleTensor)).item()
                train_losses.append(running_loss/len(train_loader))
                test_losses.append(test_loss/len(val_loader))                    
                print(f"Epoch {epoch+1}/{NUM_EPOCHS}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Val loss: {test_loss/len(val_loader):.3f}.. "
                      f"Val accuracy: {acc/len(val_loader):.3f}")
                running_loss = 0
                model.train()
                


if __name__ == "__main__":
    main()

25557032
Epoch 1/200.. Train loss: 34.083.. Val loss: 487622.337.. Val accuracy: 0.961
Epoch 1/200.. Train loss: 6.357.. Val loss: 9.428.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 5.240.. Val loss: 3.956.. Val accuracy: 0.000
Epoch 1/200.. Train loss: 3.232.. Val loss: 6.622.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 2.328.. Val loss: 5.383.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 1.806.. Val loss: 9.670.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 1.448.. Val loss: 8.017.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 1.367.. Val loss: 6.664.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 1.229.. Val loss: 14.128.. Val accuracy: 0.002
Epoch 1/200.. Train loss: 1.056.. Val loss: 9.220.. Val accuracy: 0.002
Epoch 2/200.. Train loss: 1.557.. Val loss: 34.215.. Val accuracy: 0.004
Epoch 2/200.. Train loss: 1.062.. Val loss: 8.771.. Val accuracy: 0.002
Epoch 2/200.. Train loss: 1.046.. Val loss: 23.218.. Val accuracy: 0.002
Epoch 2/200.. Train loss: 0.942.. Val loss: 26